In [1]:
import pandas as pd

In [2]:
df1=pd.read_csv('RealNewsContent.csv')

In [3]:
df2=pd.read_csv('FakeNewsContent.csv')

In [4]:
df3=pd.read_csv('politifact_fake_full.csv')

In [5]:
df4=pd.read_csv('politifact_real_full.csv')

In [6]:
df5=pd.read_csv('politifact_fixed.csv')

In [7]:
df1.head()

,text,url,type
0,"On Saturday, September 17 at 8:30 pm EST, an e...",http://eaglerising.com/36942/another-terrorist...,0
1,Less than a day after protests over the police...,http://abcn.ws/2d4lNn9,0
2,"Obama To UN: ‘Giving Up Liberty, Enhances Secu...",http://rightwingnews.com/barack-obama/obama-un...,0
3,Getty Images Wealth Of Nations Trump vs. Clint...,http://politi.co/2de2qs0,0
4,President Obama today vetoed a bill that would...,http://abcn.ws/2dh2NFs,0


In [8]:
df3.head()

,url,content
0,https://speedtalk.com/forum/viewtopic.php?t=51650,There was an error with this page
1,https://politics2020.info/index.php/2018/03/13...,There was an error with this page
2,https://www.nscdscamps.org/blog/category/paren...,There was an error with this page
3,https://howafrica.com/oscar-pistorius-attempts...,There was an error with this page
4,http://washingtonsources.org/trump-votes-for-d...,There was an error with this page


In [9]:
urls=[]
for url in df1["url"]:
    if url in df4["url"]:
        urls.append(url)


In [10]:
urls

[]

In [11]:
urls_real=[]
for url in df2["url"]:
    if url in df3["url"]:
        urls_real.append(url)


In [12]:
urls_real

[]

In [13]:
df_buzzfeed=pd.concat([df2,df1])

In [14]:
!pip install transformers


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

C:\Users\petar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
model = TFAutoModelForSequenceClassification.from_pretrained("PetarKal/Fake_News_Bert_Model2")

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at PetarKal/Fake_News_Bert_Model2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [17]:
df_buzzfeed.head()

,text,url,type
0,I woke up this morning to find a variation of ...,http://www.addictinginfo.org/2016/09/19/proof-...,1
1,Former President Bill Clinton and his Clinton ...,http://eaglerising.com/36899/charity-clinton-f...,1
2,After collapsing just before trying to step in...,http://eaglerising.com/36880/a-hillary-clinton...,1
3,"Donald Trump is, well, deplorable. He’s sugges...",http://www.addictinginfo.org/2016/09/19/trumps...,1
4,Website is Down For Maintenance,http://www.proudcons.com/clinton-foundation-ca...,1


In [18]:
df_buzzfeed.drop(columns=["url"], inplace=True)

In [19]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [20]:
df_buzfeed=df_buzzfeed.dropna()

In [21]:
def preprocess_function(examples):
    if isinstance(examples["text"], str):
        return tokenizer(examples["text"], truncation=True)
    else:
        return tokenizer("N/A", truncation=True)

In [22]:
df_buzzfeed.head()

,text,type
0,I woke up this morning to find a variation of ...,1
1,Former President Bill Clinton and his Clinton ...,1
2,After collapsing just before trying to step in...,1
3,"Donald Trump is, well, deplorable. He’s sugges...",1
4,Website is Down For Maintenance,1


In [23]:
from datasets import Dataset

dataset = Dataset.from_pandas(df_buzzfeed,preserve_index=False)


In [24]:
df_tokenized=dataset.map(preprocess_function)

Map: 100%|██████████| 182/182 [00:00<00:00, 450.24 examples/s]


In [25]:
x=df_tokenized.remove_columns(["text","type"])
y=df_buzzfeed["type"]

In [26]:
def preprocess(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors="tf")

inputs = preprocess(df_buzzfeed['text'].tolist())

In [27]:
outputs = model(**inputs)

In [28]:
import tensorflow as tf
probs = tf.nn.softmax(outputs.logits, axis=-1)

# Get the predicted class (0 or 1 for binary classification)
predictions = tf.argmax(probs, axis=1)

# Convert predictions back to a numpy array (if needed)
predictions = predictions.numpy()


In [29]:
len(predictions)

182

In [30]:
df_buzzfeed["predicted_label"]=predictions

In [31]:
df_buzzfeed

,text,type,predicted_label
0,I woke up this morning to find a variation of ...,1,1
1,Former President Bill Clinton and his Clinton ...,1,1
2,After collapsing just before trying to step in...,1,1
3,"Donald Trump is, well, deplorable. He’s sugges...",1,1
4,Website is Down For Maintenance,1,1
...,...,...,...
86,"Like much of the mainstream media, the Washing...",0,1
87,2016 Obama’s team isn’t laughing at Trump anym...,0,1
88,"Story highlights Trump has 45%, Clinton 42% an...",0,0
89,There may be a few women out there who enjoy a...,0,1


In [32]:
correct_predictions = (df_buzzfeed['predicted_label'] == df_buzzfeed['type']).sum()
total = len(df_buzzfeed)

accuracy = correct_predictions / total
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 58.79%


In [33]:
id2label = {0: "Real", 1: "Fake"}
label2id = {"Real": 0, "Fake": 1}

In [34]:
model2 = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [35]:
outputs2 = model2(**inputs)

In [36]:
import tensorflow as tf
probs2 = tf.nn.softmax(outputs2.logits, axis=-1)

# Get the predicted class (0 or 1 for binary classification)
predictions2 = tf.argmax(probs2, axis=1)

# Convert predictions back to a numpy array (if needed)
predictions2 = predictions2.numpy()


In [37]:
df_buzzfeed["predicted_label2"]=predictions2


In [38]:
correct_predictions = (df_buzzfeed['predicted_label2'] == df_buzzfeed['type']).sum()
total = len(df_buzzfeed)

accuracy = correct_predictions / total
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 50.00%


In [39]:
df_buzzfeed

,text,type,predicted_label,predicted_label2
0,I woke up this morning to find a variation of ...,1,1,1
1,Former President Bill Clinton and his Clinton ...,1,1,1
2,After collapsing just before trying to step in...,1,1,1
3,"Donald Trump is, well, deplorable. He’s sugges...",1,1,1
4,Website is Down For Maintenance,1,1,1
...,...,...,...,...
86,"Like much of the mainstream media, the Washing...",0,1,1
87,2016 Obama’s team isn’t laughing at Trump anym...,0,1,1
88,"Story highlights Trump has 45%, Clinton 42% an...",0,0,1
89,There may be a few women out there who enjoy a...,0,1,1


In [41]:
tf.nn.softmax(outputs.logits, axis=-1)

<tf.Tensor: shape=(182, 2), dtype=float32, numpy=
array([[-0.8104825 ,  0.95087457],
       [-0.8480685 ,  0.9906671 ],
       [-0.70046544,  0.77301526],
       [-0.8926599 ,  0.986966  ],
       [-0.3132396 ,  0.5002792 ],
       [ 0.699223  , -0.61473733],
       [-0.29215458,  0.3827525 ],
       [-0.86452186,  1.0040036 ],
       [-0.8812918 ,  1.0156921 ],
       [-0.39938056,  0.5869026 ],
       [ 1.2229482 , -1.0473315 ],
       [-0.7194406 ,  0.8729996 ],
       [-0.70046544,  0.77301526],
       [-0.7747539 ,  0.92918193],
       [-0.2747108 ,  0.42761746],
       [-0.88470733,  0.9978813 ],
       [ 0.48548952, -0.3728655 ],
       [ 1.004058  , -0.86361015],
       [-0.8847934 ,  0.9861777 ],
       [-0.63860416,  0.776144  ],
       [-0.73072433,  0.89198816],
       [-0.2837825 ,  0.40107644],
       [-0.9063275 ,  1.0523237 ],
       [-1.0038669 ,  1.100267  ],
       [-1.0305809 ,  1.1367226 ],
       [-1.0139432 ,  1.1209657 ],
       [-0.63412344,  0.70569223],
     